In [1]:
import os
import pandas as pd
import numpy as np
import tqdm

In [3]:
# working directory
print(os.getcwd())
# data folder
data_path = os.getcwd() + '\\' + 'data' + '\\'
print(data_path)

C:\Users\Jose Alvarez\Documents\Projects\CounterfactualSituationTesting
C:\Users\Jose Alvarez\Documents\Projects\CounterfactualSituationTesting\data\


### data1: German Credit (revised)

Consider the revised German credit by Gromping (2019), http://www1.beuth-hochschule.de/FB_II/reports/Report-2019-004.pdf, stored also by UCI: https://archive.ics.uci.edu/ml/datasets/South+German+Credit+%28UPDATE%29#.

For the original version see, e.g., https://online.stat.psu.edu/stat857/node/215/ with the following descriptions: https://online.stat.psu.edu/stat857/node/222/. Also see https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data) for more details on the dataset.

For other credit scoring datasets, see: https://github.com/kozodoi/Fair_Credit_Scoring/blob/main/data/README.md.

In [4]:
data1 = pd.read_csv(data_path + 'raw\\' + 'SouthGermanCredit\\' + 'south_formatted.csv', index_col=False, sep=" ")
print(data1.shape)
print(data1.columns.tolist())
data1.head(5)

(1000, 21)
['laufkont', 'laufzeit', 'moral', 'verw', 'hoehe', 'sparkont', 'beszeit', 'rate', 'famges', 'buerge', 'wohnzeit', 'verm', 'alter', 'weitkred', 'wohn', 'bishkred', 'beruf', 'pers', 'telef', 'gastarb', 'kredit']


,laufkont,laufzeit,moral,verw,hoehe,sparkont,beszeit,rate,famges,buerge,...,verm,alter,weitkred,wohn,bishkred,beruf,pers,telef,gastarb,kredit
0,1,18,4,2,1049,1,2,4,2,1,...,2,21,3,1,1,3,2,1,2,1
1,1,9,4,0,2799,1,3,2,3,1,...,1,36,3,1,2,3,1,1,2,1
2,2,12,2,9,841,2,4,2,2,1,...,1,23,3,1,1,2,2,1,2,1
3,1,12,4,0,2122,1,3,3,3,1,...,1,39,3,1,2,2,1,1,1,1
4,1,12,4,0,2171,1,3,4,3,1,...,2,38,1,2,2,2,2,1,1,1


In [54]:
# from English to German
rename_data1 = {'laufkont': 'status',
               'laufzeit': 'duration',
               'moral': 'credit_history',
               'verw': 'purpose',
               'hoehe': 'amount',
               'sparkont': 'savings',
               'beszeit': 'employment_duration',
               'rate': 'installment_rate',
               'famges': 'personal_status_sex',
               'buerge': 'other_debtors',
               'wohnzeit': 'present_residence',
               'verm': 'property',
               'alter': 'age',
               'weitkred': 'other_installment_plans',
               'wohn': 'housing',
               'bishkred': 'number_credits',
               'beruf': 'job',
               'pers': 'people_liable',
               'telef': 'telephone',
               'gastarb': 'foreign_worker',
               'kredit': 'credit_risk'}
# rename data1
data1.rename(columns=rename_data1, inplace=True)
print(data1.columns)

Index(['status', 'duration', 'credit_history', 'purpose', 'amount', 'savings',
       'employment_duration', 'installment_rate', 'personal_status_sex',
       'other_debtors', 'present_residence', 'property', 'age',
       'other_installment_plans', 'housing', 'number_credits', 'job',
       'people_liable', 'telephone', 'foreign_worker', 'credit_risk'],
      dtype='object')


In [55]:
# personal_status_sex 
# --- male : divorced/separated 1 
# --- female : non-single or male : single 2 
# --- male : married/widowed 3 
# --- female : single 4

def g(x):
    if x==1 or x==3:
        return 'Male'
    if x==2 or x==4:
        return 'Female'
    return x

# check
print(data1[(data1['personal_status_sex']==1) | (data1['personal_status_sex']==3)].shape[0])
# create 'gender' (our protected attribute)
data1['gender'] = data1['personal_status_sex'].map(g)
print(data1[data1['gender']=='Male'].shape[0])
# drop original col
data1 = data1.drop(columns=['personal_status_sex'])

598
598


In [56]:
# data1['id'] = data1.index +1
# data1.head(5)

In [57]:
data1.to_csv(data_path + '\\' + 'clean_GermanCreditData.csv', 
             sep='|', 
             index=False)

### data2: Law School

Taken from https://github.com/mkusner/counterfactual-fairness.

Orginal study and origin of data: https://archive.lawschooltransparency.com/reform/projects/investigations/2015/documents/NLBPS.pdf.

The Law School Admission Council conducted a survey across 163 law schools in the United States. It contains information on 21,790 law students such as their entrance exam scores (LSAT), their grade-point average (GPA) collected prior to law school, and their first year average grade (FYA).

Y: FYA; X: LSAT, GPA; A: race, sex, region (potentially)

Notes:

(1) region and sander_index are ignored in paper. Never mentioned why;

(2) first_pf is used for splitting the train and test sets. Never mentioned why in paper or code;

(3) sex==1 is for women and sex==2 for men;

(4) FYA seems normalized already (probably why they didn't use region in the logit). Never mentioned in paper or code.

Note: unclear what First Year Average (FYA) means... I would instead create a decision to accept/deny the candidate bates on a combination of their GDP and LSAT scores, which is probably what happens in real life, no?

Keep in mind that the LSAT ranges from 120 - 180 today, but from 1981 to 1991 it used a 48-point scale, ranging from 10 to 48 (as in the data...). 

Further, we can use the stats from Law Schools to mimic a decision on a candidate :). See: https://www.ilrg.com/. We could, e.g., grab Harvard's incoming class distribution and see indirect discrimination. 

In [5]:
data2 = pd.read_csv(data_path + 'raw\\' + 'law_data.csv', index_col=False, )
print(data2.shape)
print(data2.columns.tolist())
data2.head(5)

(21791, 9)
['Unnamed: 0', 'race', 'sex', 'LSAT', 'UGPA', 'region_first', 'ZFYA', 'sander_index', 'first_pf']


,Unnamed: 0,race,sex,LSAT,UGPA,region_first,ZFYA,sander_index,first_pf
0,0,White,1,39.0,3.1,GL,-0.98,0.782738,1.0
1,1,White,1,36.0,3.0,GL,0.09,0.735714,1.0
2,2,White,2,30.0,3.1,MS,-0.35,0.670238,1.0
3,5,Hispanic,2,39.0,2.2,NE,0.58,0.697024,1.0
4,6,White,1,37.0,3.4,GL,-1.26,0.786310,1.0


In [59]:
data2.groupby('region_first').count()

,Unnamed: 0,race,sex,LSAT,UGPA,ZFYA,sander_index,first_pf
region_first,,,,,,,,
FW,2904,2904,2904,2904,2904,2904,2904,2904
GL,3822,3822,3822,3822,3822,3822,3822,3822
MS,2346,2346,2346,2346,2346,2346,2346,2346
MW,1071,1071,1071,1071,1071,1071,1071,1071
Mt,1147,1147,1147,1147,1147,1147,1147,1147
NE,4302,4302,4302,4302,4302,4302,4302,4302
NG,1133,1133,1133,1133,1133,1133,1133,1133
NW,163,163,163,163,163,163,163,163
PO,1,1,1,1,1,1,1,1


In [60]:
# drop 'PO' like in Kusner et al.
data2 = data2[data2['region_first']!='PO']
data2.reset_index(drop=True, inplace=True)

In [61]:
data2.groupby('region_first').count()

,Unnamed: 0,race,sex,LSAT,UGPA,ZFYA,sander_index,first_pf
region_first,,,,,,,,
FW,2904,2904,2904,2904,2904,2904,2904,2904
GL,3822,3822,3822,3822,3822,3822,3822,3822
MS,2346,2346,2346,2346,2346,2346,2346,2346
MW,1071,1071,1071,1071,1071,1071,1071,1071
Mt,1147,1147,1147,1147,1147,1147,1147,1147
NE,4302,4302,4302,4302,4302,4302,4302,4302
NG,1133,1133,1133,1133,1133,1133,1133,1133
NW,163,163,163,163,163,163,163,163
SC,2251,2251,2251,2251,2251,2251,2251,2251


In [62]:
data2 = data2.drop(columns=['Unnamed: 0', 'sander_index', 'first_pf', 'region_first'])
data2['sex'] = data2['sex'].map({1: 'Female', 2: 'Male'})
data2.head(5)

,race,sex,LSAT,UGPA,ZFYA
0,White,Female,39.0,3.1,-0.98
1,White,Female,36.0,3.0,0.09
2,White,Male,30.0,3.1,-0.35
3,Hispanic,Male,39.0,2.2,0.58
4,White,Female,37.0,3.4,-1.26


In [63]:
data2.groupby('race').count()

,sex,LSAT,UGPA,ZFYA
race,,,,
Amerindian,99,99,99,99
Asian,845,845,845,845
Black,1282,1282,1282,1282
Hispanic,488,488,488,488
Mexican,389,389,389,389
Other,293,293,293,293
Puertorican,110,110,110,110
White,18284,18284,18284,18284


In [64]:
# In Situtation Testing we want to go from white to non-white (no?)
def r_nonwhite(x):
    if x=='White':
        return 'White'
    else:
        return 'NonWhite'

def r_simpler(x):
    if x=='Hispanic' or x=='Mexican' or x=='Puertorican':
        return 'Latino'
    if x=='White':
        return 'White'
    if x=='Amerindian' or x=='Other':
        return 'Other'
    if x=='Black':
        return 'Black'
    if x=='Asian':
        return 'Asian'


In [65]:
data2['race_nonwhite'] = data2['race'].map(r_nonwhite)

data2['race_simpler']  = data2['race'].map(r_simpler)

In [66]:
data2.groupby('race_nonwhite').count()

,race,sex,LSAT,UGPA,ZFYA,race_simpler
race_nonwhite,,,,,,
NonWhite,3506,3506,3506,3506,3506,3506
White,18284,18284,18284,18284,18284,18284


In [67]:
data2.groupby('race_simpler').count()

,race,sex,LSAT,UGPA,ZFYA,race_nonwhite
race_simpler,,,,,,
Asian,845,845,845,845,845,845
Black,1282,1282,1282,1282,1282,1282
Latino,987,987,987,987,987,987
Other,392,392,392,392,392,392
White,18284,18284,18284,18284,18284,18284


In [68]:
# data2['id'] = data2.index + 1
# data2.head(5)

In [69]:
data2.to_csv(data_path + '\\' + 'clean_LawData.csv', sep='|', index=False)

### data3: Adult UCI

Taken from folktables: https://github.com/zykls/folktables.

In [ ]:
# TODO... explore and retrieve Adult Income dataset

In [ ]:
#
# EOF
#

# data = arff.loadarff(os.getcwd() + '\\data' + '\\dutch_census_2001.arff')
# data
# df = pd.DataFrame(data[0])
# df.head(10)